In [1]:
%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
# Loading your IBM Q account(s)
provider = IBMQ.load_account()

/opt/conda/lib/python3.7/site-packages/qiskit/providers/ibmq/ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


In [2]:
IBMQ.get_provider(hub='ibm-q')
IBMQ.get_provider(project='my_project')

IBMQProviderError: 'No provider matches the specified criteria: hub = None, group = None, project = my_project'

In [3]:
provider = IBMQ.get_provider(group='open')
provider.backends()


[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_vigo') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_ourense') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_valencia') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_athens') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_santiago') from IBMQ(hub='ibm-q', group='open', project='main')>]

In [4]:
#Filter the list of backends to include only non-simulator and operational
provider.backends(simulator=False, operational=True)

[<IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_vigo') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_ourense') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_valencia') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_athens') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_santiago') from IBMQ(hub='ibm-q', group='open', project='main')>]

In [6]:
backend = provider.get_backend('ibmq_santiago')

In [7]:
#Import least busy function
from qiskit.providers.ibmq import least_busy
# get least busy devices which are less than 6 qubits and not a simulator
small_devices = provider.backends(filters=lambda 
                                  x: x.configuration().n_qubits < 6 and not x.configuration().simulator)

# get least busy devices which are larger than 6 qubits and not a simulator
large_devices = provider.backends(filters=lambda 
                                  x: x.configuration().n_qubits > 6 and not x.configuration().simulator)

print('The least busy small devices: {}'.format(least_busy(small_devices)))
print('The least busy large devices: {}'.format(least_busy(large_devices)))

The least busy small devices: ibmq_armonk
The least busy large devices: ibmq_16_melbourne


In [10]:
backend = provider.get_backend('ibmq_valencia')
backend.name()
status = backend.status()
jobs_in_queue = status.pending_jobs
print('Number of pending jobs in the queue: ', jobs_in_queue)

Number of pending jobs in the queue:  97


In [11]:
backend.configuration()
backend

<IBMQBackend('ibmq_valencia') from IBMQ(hub='ibm-q', group='open', project='main')>

In [12]:
#Display the number of qubits from the backend properties
backend.properties().qubits

[[Nduv(datetime.datetime(2020, 10, 25, 8, 13, 30, tzinfo=tzlocal()), T1, us, 99.45878134641065),
  Nduv(datetime.datetime(2020, 10, 25, 8, 15, tzinfo=tzlocal()), T2, us, 60.53505457610266),
  Nduv(datetime.datetime(2020, 10, 25, 8, 38, 18, tzinfo=tzlocal()), frequency, GHz, 4.74389238032289),
  Nduv(datetime.datetime(2020, 10, 25, 8, 38, 18, tzinfo=tzlocal()), anharmonicity, GHz, -0.31386048358781926),
  Nduv(datetime.datetime(2020, 10, 25, 8, 13, 6, tzinfo=tzlocal()), readout_error, , 0.03859999999999997),
  Nduv(datetime.datetime(2020, 10, 25, 8, 13, 6, tzinfo=tzlocal()), prob_meas0_prep1, , 0.0514),
  Nduv(datetime.datetime(2020, 10, 25, 8, 13, 6, tzinfo=tzlocal()), prob_meas1_prep0, , 0.0258)],
 [Nduv(datetime.datetime(2020, 10, 25, 8, 13, 30, tzinfo=tzlocal()), T1, us, 85.08507838611285),
  Nduv(datetime.datetime(2020, 10, 25, 8, 16, 30, tzinfo=tzlocal()), T2, us, 43.634862188284025),
  Nduv(datetime.datetime(2020, 10, 25, 8, 38, 18, tzinfo=tzlocal()), frequency, GHz, 4.6608700762

In [14]:
# Print out the frequency of qubit (0)
print('Frequency of first qubit is: '+ str(backend.properties().frequency(0)))
#print out the readout error of qubit (0)
print('Readout error of first qubit is: '+ str(backend.properties().readout_error(0)))

Frequency of first qubit is: 4743892380.32289
Readout error of first qubit is: 0.03859999999999997


In [16]:
#Run a few jobs on this backend to generate jobs on the backend
qc = QuantumCircuit(1,1)
qc.h(0)
qc.measure_all()
for i in range(0,3):
    result = execute(qc, backend, shots=1024).result()

In [18]:
for executed_job in backend.jobs(limit=3):
    print('Job id: '
         + str(executed_job.job_id()) + ', '
         + str(executed_job.end_date) + ', '
         + str(executed_job.status()))

AttributeError: Attribute end_date is not defined.